In [24]:
import requests
import os
from fastai.vision.all import *

from duckduckgo_search import DDGS

base_path="data"
tmp_path='tmp'

def get_images(keywords, count):
    results = DDGS().images(
        keywords=keywords,
        max_results=count,
    )
    image_urls = [result['image'] for result in results]  # List comprehension
    return image_urls

In [ ]:
# download dogs images
dogs = get_images("dogs", 100)
puppies = get_images("puppies", 100)
download_images(f"{base_path}/dogs", null, dogs + puppies)

In [ ]:
# download cats images
cats = get_images("cats", 100)
kittens = get_images("kittens", 100)
all = [*cats, *kittens]
download_images(f"{base_path}/cats", null, all)

In [ ]:
# resize images
resize_images(f"{base_path}/dogs", max_size=400, dest=f"{base_path}/dogs")
resize_images(f"{base_path}/cats", max_size=400, dest=f"{base_path}/cats")

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(base_path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate) #using resnet18 pretrained model
learn.fine_tune(3)

In [ ]:
# prepare test data
def copy_and_rename_images(source_folder, destination_folder):
    source_path = Path(source_folder)
    destination_path = Path(destination_folder)
    
    # Create destination folder if it does not exist
    destination_path.mkdir(parents=True, exist_ok=True)
    
    # Get all image files including those in subfolders
    image_files = get_image_files(source_path)
    
    # Copy and rename images
    for i, file in enumerate(image_files):
        parent_folder_name = file.parent.name
        new_filename = f"{parent_folder_name}_{i+1}{file.suffix}"
        shutil.copy(file, destination_path / new_filename)

copy_and_rename_images(base_path, tmp_path) # copy files to tmp folder

In [25]:
#remove failed images
failed = verify_images(get_image_files(tmp_path))
failed.map(Path.unlink)
print(len(failed))

11


In [30]:
randoms = get_files(tmp_path)  # Assuming this returns a list of Image objects

for image in randoms:
    label_prediction, _, probs = learn.predict(image) 
    if label_prediction not in str(image): 
        print(f"Label: {label_prediction}. Filename: {image}") 


Label: dogs. Filename: tmp/cats_150.jpg


Label: dogs. Filename: tmp/cats_152.jpg


Label: dogs. Filename: tmp/cats_154.jpg


Label: dogs. Filename: tmp/cats_137.jpg


Label: dogs. Filename: tmp/cats_264.jpg
